In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
import os

c:\Users\royho\anaconda3\envs\myFastapi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dotenv import load_dotenv

load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [5]:
text = """How do emotions influence decision-making processes in individuals, and to what extent do external factors contribute to shaping emotional responses?
Emotions play a pivotal role in human decision-making, acting as powerful influencers that guide choices and actions. The intricate interplay between emotions and decision-making processes is a fascinating area of study that delves into the depths of psychology, neuroscience, and behavioral economics. Understanding the nuances of this relationship requires exploration into the mechanisms through which emotions arise, how they are processed, and the extent to which external factors contribute to shaping emotional responses.
At the core of emotional influence on decision-making lies the limbic system, a complex network of brain structures responsible for processing emotions. The amygdala, in particular, plays a crucial role in the emotional responses associated with decision-making. This almond-shaped structure is responsible for processing and interpreting emotional stimuli, triggering physiological responses that prepare the body for action. Consequently, the emotional signals generated by the amygdala can significantly impact the decision-making process, either enhancing or impairing cognitive functions.
The impact of emotions on decision-making is not limited to the neural level; it extends to the psychological and behavioral dimensions as well. Emotions can serve as valuable guides, providing individuals with insights into their preferences, values, and potential risks associated with different choices. Positive emotions, such as joy or excitement, may lead to more risk-taking behavior, while negative emotions, like fear or sadness, can encourage cautious decision-making.
However, the relationship between emotions and decision-making is not a one-way street. External factors, such as social, cultural, and environmental influences, also contribute significantly to shaping emotional responses. Social context, for instance, can modulate emotional experiences, with individuals often conforming to societal norms and expectations. Cultural differences further highlight the variability in emotional expression and its impact on decision-making across diverse populations.
Moreover, environmental stimuli can trigger emotional responses that, in turn, influence decision-making. The concept of environmental priming suggests that the physical environment can activate specific emotional states, subsequently influencing behavior. For instance, a well-lit and aesthetically pleasing environment may promote positive emotions, potentially leading to more optimistic decision-making.
The influence of external factors on emotional responses raises questions about the autonomy of individual decision-making. To what extent are decisions truly reflective of personal preferences and rational considerations, and how much do external influences shape the emotional landscape that guides these decisions? Unraveling this intricate web of interactions requires a comprehensive examination of both internal and external contributors to the decision-making process.
Additionally, the role of individual differences in emotional regulation and perception cannot be overlooked. People vary in their ability to regulate emotions, with factors such as personality, upbringing, and past experiences shaping emotional resilience. Understanding how these individual differences intersect with external influences can provide valuable insights into the variability observed in decision-making across diverse populations.
In conclusion, the relationship between emotions and decision-making is a multifaceted and dynamic interplay influenced by neural, psychological, and external factors. Exploring the intricate mechanisms through which emotions shape decisions and understanding the extent to which external influences contribute to emotional responses are essential for gaining a holistic perspective on human behavior. This exploration not only enhances our understanding of decision-making processes but also has implications for fields ranging from psychology and neuroscience to economics and public policy."""



In [6]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text(text)
    return chunks

In [7]:
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

In [8]:
def get_conversational_chain():

    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    
    memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt, memory=memory)

    return chain

In [9]:
def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
    
    new_db = FAISS.load_local("faiss_index", embeddings)
    docs = new_db.similarity_search(user_question)
    

    chain = get_conversational_chain()
    response = chain(
        {"input_documents":docs, "question": user_question, "history": "history"}
        , return_only_outputs=True)

    return response

In [10]:
text_chunks = get_text_chunks(text)
get_vector_store(text_chunks)

# user_input("where is canada?")

In [11]:
user_input("How do emotions influence decision-making processes in individuals ?")

{'output_text': 'Emotions play a pivotal role in human decision-making, acting as powerful influencers that guide choices and actions. The intricate interplay between emotions and decision-making processes is a fascinating area of study that delves into the depths of psychology, neuroscience, and behavioral economics. Understanding the nuances of this relationship requires exploration into the mechanisms through which emotions arise, how they are processed, and the extent to which external factors contribute to shaping emotional responses.\n\nThe impact of emotions on decision-making is not limited to the neural level; it extends to the psychological and behavioral dimensions as well. Emotions can serve as valuable guides, providing individuals with insights into their preferences, values, and potential risks associated with different choices. Positive emotions, such as joy or excitement, may lead to more risk-taking behavior, while negative emotions, like fear or sadness, can encourag